# Hypothesis 1

## Task

Vectorize each instance of the roof side based on the aerial image.

## Assumptions for hypothesis 1
1. It is possible to detect each instance of the roof side separately;
2. It is possible to group separate roof polygons into one bulding accurately.

## Model

CNN takes as input a city tile and extracts keypoints and edge evidence of roof side footprints, which are fed sequentially to the multi-layer ConvLSTM modules. The latter produces a vector representation for each object in a given tile. 

We use a VGG-16 without tail layers as the CNN backbone to extract skip features (see Fig.1) with the size of the input image. Meanwhile, the FPN also takes features from different layers of the backbone to construct a feature pyramid and predicts multiple bounding boxes containing the roof sides. For a single roof side, with the skip feature map and its bounding box, followed by RoIAlign, the local features F are obtained.

<img src="Fig1.png">

**Fig. 1 Skip features**. *Refinement module R is responsible for inverting the effect of pooling and doubling the resolution of the input mask encoding. Each module takes as input a mask encoding M<sub>i</sub> generated in the top-down pass, along
with matching features F<sub>i</sub> generated in the bottom-up pass, and learns to merge the information to generate a new upsampled object encoding M<sub>i</sub>+1. In other words: M<sub>i</sub>+1 = Ri (M<sub>i</sub>, F<sub>i</sub>). Multiple such modules are stacked (one module per pooling layer). The final output of our network is a pixel labeling of the same resolution as the input image.* 

<img src="Fig2.png">

**Fig. 2 RoIAlign.** *RoIAlign is an operation for extracting a small feature map from each RoI. To avoid any quantization of the RoI boundaries or bins, RoIAlign uses bilinear interpolation to compute the exact values of the input features at four regularly sampled locations in each RoI bin, and the result is then aggregated (using max or average).*

<img src="Fig3.png">

**Fig. 3. CNN design.** *We apply convolutional layers to the feature in order to generate a heat-map mask of building boundaries B that delineate the object of interest. This is followed by additional convolutional layers outputting a mask of candidate keypoints, denoted by V . Both B and V have a sizenequal to $\frac{1}{8}$ the size of the input image. Among all candidate keypoints, we select those w points with the highest scorenin V as starting point y<sub>0</sub>.*

<img src="Fig4.png">

**Fig. 4. RNN design** *RNN outputs y<sub>t</sub>’s potential location P(y<sub>t+1</sub>|y<sub>t</sub>, y<sub>t-1</sub>, y<sub>0</sub>) at each step t. We input both, y<sub>t</sub> and y<sub>t-1</sub> to compute the conditional probability distribution of y<sub>t+1</sub> because it allows defining a unique direction. If given two neighboring vertices with an order in a polygon, the next vertex in this polygon is uniquely determined. Note that the distribution also involves the end signal <eos> (end of sequence), which indicates that the polygon reaches a closed shape and the prediction procedure should come to the end. The final, end vertex in a polygon thus corresponds to the very first, starting vertex y<sub>0</sub>, which therefore has to be included at each step.
In practice, we ultimately concatenate F, B, V , y<sub>0</sub>  and feed the resulting tensor to a multi-layer RNN with ConvLSTM cells in order to sequentially predict the vertices that will delineate the object of interest, until it predicts the <eos> symbol.Then we simply connect all sequentially predicted vertices to obtain the final building polygon.*

## Training

We set the model parameters using size 28×28 for F, B, V and y<sub>t</sub>, and set the number of layers of the RNN to 3. The maximum length of a sequence when training is set to be 30. The total loss is a combined loss from the FPN, CNN and RNN parts. The FPN loss consists of a crossentropy loss for anchor classification and a smooth L1 loss for anchor regression. The CNN loss refers to the log loss for the mask of boundary and vertices, and the RNN loss is the cross-entropy loss for the multi-class classification at each time step. For training, we use the Adam optimizer with batch size 4 and an initial learning rate of 0.0001, as well as default β1 and β2.

## Inference 

In the inference phase, we use beam search with a width 5. We select top w vertices with highest probability in V as the starting vertices, then followed by a general beam search procedure. Among the w polygon candidates, we choose the one with the highest probability as the output.

## Evaluation

For building extraction, we report the standard MS COCO measures including average precision (AP, averaged over IoU thresholds), AP50, AP75 and APS, APM, APL (AP at different scales). To measure the proportion of buildings detected by our approach with respect to the ground truth, we additionally evaluate average recall (AR), which is not commonly used in previous works such as. Both AP and AR are evaluated using mask IoU. However, we would like to emphasize that in contrast to pixel-wise output masks produced by common methods for building footprint extraction, our outputs are polygon representations of building footprints.

**Average Precision (AP):**
* AP = % AP at IoU=.50:.05:.95 (primary challenge metric)

* AP<sup>IoU=.50</sup> = % AP at IoU=.50 (PASCAL VOC metric)

* AP<sup>IoU=.75</sup> = % AP at IoU=.75 (strict metric)

**AP Across Scales:**
* AP<sup>small</sup> = % AP for small objects: area < 32<sup>2</sup>

* AP<sup>medium</sup> = % AP for medium objects: 32<sup>2</sup> < area < 96<sup>2</sup> 
* AP<sup>large</sup> = % AP for large objects: area > 96<sup>2</sup>

**Average Recall (AR):**
* AR<sup>max=1</sup> = % AR given 1 detection per image
* AR<sup>max=10</sup> = % AR given 10 detections per image
* AR<sup>max=100</sup> = % AR given 100 detections per image

**AR Across Scales:**
* AR<sup>small</sup> = % AR for small objects: area < 32<sup>2</sup>
* AR<sup>medium</sup> = % AR for medium objects: 32<sup>2</sup> < area < 96<sup>2</sup>
* AR<sup>large</sup> = % AR for large objects: area > 96<sup>2</sup>